In [6]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


In [11]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.4990,  0.3689,  0.1628,  0.2991,  0.0042,  0.1901,  0.3264, -0.1097,
         -0.1813,  0.1229,  0.3764, -0.4976, -0.1973,  0.5604, -0.1673,  0.1547,
         -0.1578,  0.1146, -0.4592,  0.1786],
        [ 0.0688,  0.2028, -0.0027,  0.0083, -0.0386,  0.3479,  0.2187,  0.1520,
         -0.3219, -0.2599,  0.1397,  0.0586, -0.4167,  0.3415, -0.1757, -0.0437,
          0.0054, -0.1790, -0.4541,  0.5466],
        [ 0.0932,  0.5244,  0.2015,  0.2086,  0.0611, -0.1054,  0.4648,  0.3157,
         -0.0880,  0.0845,  0.0534, -0.0690, -0.2042,  0.3420, -0.2397,  0.1432,
         -0.1493,  0.2464, -0.2920,  0.4249]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.4990, 0.3689, 0.1628, 0.2991, 0.0042, 0.1901, 0.3264, 0.0000, 0.0000,
         0.1229, 0.3764, 0.0000, 0.0000, 0.5604, 0.0000, 0.1547, 0.0000, 0.1146,
         0.0000, 0.1786],
        [0.0688, 0.2028, 0.0000, 0.0083, 0.0000, 0.3479, 0.2187, 0.1520, 0.0000,
         0.0000, 0.1397, 0.0586, 0.0000, 0.3415, 0.000

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0176,  0.0078, -0.0261,  ...,  0.0104,  0.0288, -0.0092],
        [ 0.0117,  0.0031,  0.0005,  ..., -0.0291,  0.0132,  0.0148]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0288, -0.0309], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0148,  0.0385,  0.0267,  ...,  0.0214,  0.0223,  0.0414],
        [-0.0252, -0.0417, -0.0090,  ...,  0.0346, -0.0180,  0.0048]],
       device='cuda:0', grad_fn=<Slic